# Vexpresso Walkthrough

## In this walkthrough, we will be using a small dataset of Pokemon statistics to showcase vexpresso's capabilities

### Imports

In [1]:
import json
import pandas as pd
from langchain.embeddings import HuggingFaceHubEmbeddings, HuggingFaceEmbeddings
import numpy as np
import daft
import vexpresso
from vexpresso.retriever import NumpyRetriever
from vexpresso import transformation

/home/shyam/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Lets look at the Data: from https://github.com/Purukitto/pokemon-data.json

In [2]:
with open("./data/pokedex.json", 'r') as f:
    stuff = json.load(f)

df = pd.DataFrame(stuff)

In [3]:
df.head(10)

,id,name,type,base,species,description,evolution,profile,image
0,1,"{'english': 'Bulbasaur', 'japanese': 'フシギダネ', ...","[Grass, Poison]","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. A...",Seed Pokémon,Bulbasaur can be seen napping in bright sunlig...,"{'next': [['2', 'Level 16']]}","{'height': '0.7 m', 'weight': '6.9 kg', 'egg':...",{'sprite': 'https://raw.githubusercontent.com/...
1,2,"{'english': 'Ivysaur', 'japanese': 'フシギソウ', 'c...","[Grass, Poison]","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. A...",Seed Pokémon,There is a bud on this Pokémon’s back. To supp...,"{'prev': ['1', 'Level 16'], 'next': [['3', 'Le...","{'height': '1 m', 'weight': '13 kg', 'egg': ['...",{'sprite': 'https://raw.githubusercontent.com/...
2,3,"{'english': 'Venusaur', 'japanese': 'フシギバナ', '...","[Grass, Poison]","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. A...",Seed Pokémon,There is a large flower on Venusaur’s back. Th...,"{'prev': ['2', 'Level 32']}","{'height': '2 m', 'weight': '100 kg', 'egg': [...",{'sprite': 'https://raw.githubusercontent.com/...
3,4,"{'english': 'Charmander', 'japanese': 'ヒトカゲ', ...",[Fire],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. A...",Lizard Pokémon,The flame that burns at the tip of its tail is...,"{'next': [['5', 'Level 16']]}","{'height': '0.6 m', 'weight': '8.5 kg', 'egg':...",{'sprite': 'https://raw.githubusercontent.com/...
4,5,"{'english': 'Charmeleon', 'japanese': 'リザード', ...",[Fire],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. A...",Flame Pokémon,Charmeleon mercilessly destroys its foes using...,"{'prev': ['4', 'Level 16'], 'next': [['6', 'Le...","{'height': '1.1 m', 'weight': '19 kg', 'egg': ...",{'sprite': 'https://raw.githubusercontent.com/...
5,6,"{'english': 'Charizard', 'japanese': 'リザードン', ...","[Fire, Flying]","{'HP': 78, 'Attack': 84, 'Defense': 78, 'Sp. A...",Flame Pokémon,Charizard flies around the sky in search of po...,"{'prev': ['5', 'Level 36']}","{'height': '1.7 m', 'weight': '90.5 kg', 'egg'...",{'sprite': 'https://raw.githubusercontent.com/...
6,7,"{'english': 'Squirtle', 'japanese': 'ゼニガメ', 'c...",[Water],"{'HP': 44, 'Attack': 48, 'Defense': 65, 'Sp. A...",Tiny Turtle Pokémon,Squirtle’s shell is not merely used for protec...,"{'next': [['8', 'Level 16']]}","{'height': '0.5 m', 'weight': '9 kg', 'egg': [...",{'sprite': 'https://raw.githubusercontent.com/...
7,8,"{'english': 'Wartortle', 'japanese': 'カメール', '...",[Water],"{'HP': 59, 'Attack': 63, 'Defense': 80, 'Sp. A...",Turtle Pokémon,"Its tail is large and covered with a rich, thi...","{'prev': ['7', 'Level 16'], 'next': [['9', 'Le...","{'height': '1 m', 'weight': '22.5 kg', 'egg': ...",{'sprite': 'https://raw.githubusercontent.com/...
8,9,"{'english': 'Blastoise', 'japanese': 'カメックス', ...",[Water],"{'HP': 79, 'Attack': 83, 'Defense': 100, 'Sp. ...",Shellfish Pokémon,Blastoise has water spouts that protrude from ...,"{'prev': ['8', 'Level 36']}","{'height': '1.6 m', 'weight': '85.5 kg', 'egg'...",{'sprite': 'https://raw.githubusercontent.com/...
9,10,"{'english': 'Caterpie', 'japanese': 'キャタピー', '...",[Bug],"{'HP': 45, 'Attack': 30, 'Defense': 35, 'Sp. A...",Worm Pokémon,"Its body is soft and weak. In nature, its perp...","{'next': [['11', 'Level 7']]}","{'height': '0.3 m', 'weight': '2.9 kg', 'egg':...",{'sprite': 'https://raw.githubusercontent.com/...


## Create Collection

In [4]:
collection = vexpresso.create(
    data="./data/pokedex.json",
)

2023-06-05 18:48:08.484 | INFO     | daft.context:runner:88 - Using PyRunner


In [5]:
collection.show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,descriptionUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64
1,"{'chinese': '妙蛙种子', 'english': 'Bulbasaur', 'french': 'Bu...","['Grass', 'Poison']","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. Attack': 65,...",Seed Pokémon,Bulbasaur can be seen napping in bright sunlight. There i...,"{'next': [['2', 'Level 16']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,0
2,"{'chinese': '妙蛙草', 'english': 'Ivysaur', 'french': 'Herbi...","['Grass', 'Poison']","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. Attack': 80,...",Seed Pokémon,There is a bud on this Pokémon’s back. To support its wei...,"{'next': [['3', 'Level 32']], 'prev': ['1', 'Level 16']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,1
3,"{'chinese': '妙蛙花', 'english': 'Venusaur', 'french': 'Flor...","['Grass', 'Poison']","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. Attack': 100...",Seed Pokémon,There is a large flower on Venusaur’s back. The flower is...,"{'next': None, 'prev': ['2', 'Level 32']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,2
4,"{'chinese': '小火龙', 'english': 'Charmander', 'french': 'Sa...",['Fire'],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. Attack': 60,...",Lizard Pokémon,The flame that burns at the tip of its tail is an indicat...,"{'next': [['5', 'Level 16']], 'prev': None}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,3
5,"{'chinese': '火恐龙', 'english': 'Charmeleon', 'french': 'Re...",['Fire'],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. Attack': 80,...",Flame Pokémon,Charmeleon mercilessly destroys its foes using its sharp ...,"{'next': [['6', 'Level 36']], 'prev': ['4', 'Level 16']}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,4


In [6]:
hf_embeddings = HuggingFaceEmbeddings()
@transformation
def hf_embedding_function(content):
    return np.array(hf_embeddings.embed_documents(content))

In [7]:
collection = collection.embed(content=list(df["description"]), column_name="description", embedding_fn=hf_embedding_function).execute()

In [8]:
collection.show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,descriptionUtf8,embeddings_descriptionPython
1,"{'chinese': '妙蛙种子', 'english': 'Bulbasaur', 'french': 'Bu...","['Grass', 'Poison']","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. Attack': 65,...",Seed Pokémon,"{'next': [['2', 'Level 16']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,0,Bulbasaur can be seen napping in bright sunlight. There i...,"<np.ndarrayshape=(768,)dtype=float64>"
2,"{'chinese': '妙蛙草', 'english': 'Ivysaur', 'french': 'Herbi...","['Grass', 'Poison']","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. Attack': 80,...",Seed Pokémon,"{'next': [['3', 'Level 32']], 'prev': ['1', 'Level 16']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,1,There is a bud on this Pokémon’s back. To support its wei...,"<np.ndarrayshape=(768,)dtype=float64>"
3,"{'chinese': '妙蛙花', 'english': 'Venusaur', 'french': 'Flor...","['Grass', 'Poison']","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. Attack': 100...",Seed Pokémon,"{'next': None, 'prev': ['2', 'Level 32']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,2,There is a large flower on Venusaur’s back. The flower is...,"<np.ndarrayshape=(768,)dtype=float64>"
4,"{'chinese': '小火龙', 'english': 'Charmander', 'french': 'Sa...",['Fire'],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. Attack': 60,...",Lizard Pokémon,"{'next': [['5', 'Level 16']], 'prev': None}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,3,The flame that burns at the tip of its tail is an indicat...,"<np.ndarrayshape=(768,)dtype=float64>"
5,"{'chinese': '火恐龙', 'english': 'Charmeleon', 'french': 'Re...",['Fire'],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. Attack': 80,...",Flame Pokémon,"{'next': [['6', 'Level 36']], 'prev': ['4', 'Level 16']}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,4,Charmeleon mercilessly destroys its foes using its sharp ...,"<np.ndarrayshape=(768,)dtype=float64>"


## Querying the collection's embeddings, lazy execution
### This doesn't run anything until `execute()` is called

In [9]:
sleepy_pokemon = collection.query("embeddings_description", query=["Loves to sleep"], k=10)
sleepy_pokemon.df

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,descriptionUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,embeddings_descriptionPython,embeddings_description_scoreFloat64


In [10]:
sleepy_pokemon.execute().show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,descriptionUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,embeddings_descriptionPython,embeddings_description_scoreFloat64
143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'french': 'Ronfl...",['Normal'],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp. Attack': 6...",Sleeping Pokémon,It has no interest in anything other than eating. Even if...,"{'next': None, 'prev': ['446', 'high Friendship']}","{'ability': [['Immunity', 'false'], ['Thick Fat', 'false'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,142,"<np.ndarrayshape=(768,)dtype=float64>",0.561686
25,"{'chinese': '皮卡丘', 'english': 'Pikachu', 'french': 'Pikac...",['Electric'],"{'HP': 35, 'Attack': 55, 'Defense': 40, 'Sp. Attack': 50,...",Mouse Pokémon,"While sleeping, it generates electricity in the sacs in i...","{'next': [['26', 'use Thunder Stone']], 'prev': ['172', '...","{'ability': [['Static', 'false'], ['Lightning Rod', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,24,"<np.ndarrayshape=(768,)dtype=float64>",0.474437
328,"{'chinese': '大颚蚁', 'english': 'Trapinch', 'french': 'Krak...",['Ground'],"{'HP': 45, 'Attack': 100, 'Defense': 45, 'Sp. Attack': 45...",Ant Pit Pokémon,"It builds nests that double as traps, lying in wait for i...","{'next': [['329', 'Level 35']], 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Arena Trap', 'f...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,327,"<np.ndarrayshape=(768,)dtype=float64>",0.450725
722,"{'chinese': '木木枭', 'english': 'Rowlet', 'french': 'Brindi...","['Grass', 'Flying']","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. Attack': 50,...",Grass Quill Pokémon,"It feels relaxed in tight, dark places and has been known...","{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Reach', 'true'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,721,"<np.ndarrayshape=(768,)dtype=float64>",0.445208
554,"{'chinese': '火红不倒翁', 'english': 'Darumaka', 'french': 'Da...",['Fire'],"{'HP': 70, 'Attack': 90, 'Defense': 45, 'Sp. Attack': 15,...",Zen Charm Pokémon,"When it sleeps, it pulls its limbs into its body and its ...","{'next': [['555', 'use Ice Stone']], 'prev': None}","{'ability': [['Hustle', 'false'], ['Inner Focus', 'true']...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,553,"<np.ndarrayshape=(768,)dtype=float64>",0.430829


### This allows for query chaining

In [11]:
sleepy_water_pokemon = collection.query("embeddings_description", query=["Loves to sleep"], k=10) \
                                 .query("embeddings_description", query=["Lives in water"], k=5) \
                                 .execute()

#### Show daft DataFrame

In [12]:
sleepy_water_pokemon.show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,descriptionUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,embeddings_descriptionPython,embeddings_description_scoreFloat64
328,"{'chinese': '大颚蚁', 'english': 'Trapinch', 'french': 'Krak...",['Ground'],"{'HP': 45, 'Attack': 100, 'Defense': 45, 'Sp. Attack': 45...",Ant Pit Pokémon,"It builds nests that double as traps, lying in wait for i...","{'next': [['329', 'Level 35']], 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Arena Trap', 'f...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,327,"<np.ndarrayshape=(768,)dtype=float64>",0.487235
127,"{'chinese': '凯罗斯', 'english': 'Pinsir', 'french': 'Scarab...",['Bug'],"{'HP': 65, 'Attack': 125, 'Defense': 100, 'Sp. Attack': 5...",Stag Beetle Pokémon,"Although it’s tough, it can’t handle cold well. When nigh...","{'next': None, 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Mold Breaker', ...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,126,"<np.ndarrayshape=(768,)dtype=float64>",0.442589
554,"{'chinese': '火红不倒翁', 'english': 'Darumaka', 'french': 'Da...",['Fire'],"{'HP': 70, 'Attack': 90, 'Defense': 45, 'Sp. Attack': 15,...",Zen Charm Pokémon,"When it sleeps, it pulls its limbs into its body and its ...","{'next': [['555', 'use Ice Stone']], 'prev': None}","{'ability': [['Hustle', 'false'], ['Inner Focus', 'true']...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,553,"<np.ndarrayshape=(768,)dtype=float64>",0.406497
722,"{'chinese': '木木枭', 'english': 'Rowlet', 'french': 'Brindi...","['Grass', 'Flying']","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. Attack': 50,...",Grass Quill Pokémon,"It feels relaxed in tight, dark places and has been known...","{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Reach', 'true'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,721,"<np.ndarrayshape=(768,)dtype=float64>",0.401396
775,"{'chinese': '树枕尾熊', 'english': 'Komala', 'french': 'Bouma...",['Normal'],"{'HP': 65, 'Attack': 115, 'Defense': 65, 'Sp. Attack': 75...",Drowsing Pokémon,It remains asleep from birth to death as a result of the ...,"{'next': None, 'prev': None}","{'ability': [['Comatose', 'false']], 'egg': ['Field'], 'g...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,774,"<np.ndarrayshape=(768,)dtype=float64>",0.371769


#### Show pandas

In [13]:
sleepy_water_pokemon.to_pandas()

,id,name,type,base,species,description,evolution,profile,image,vexpresso_index,embeddings_description,embeddings_description_score
0,328,"{'chinese': '大颚蚁', 'english': 'Trapinch', 'fre...",[Ground],"{'HP': 45, 'Attack': 100, 'Defense': 45, 'Sp. ...",Ant Pit Pokémon,"It builds nests that double as traps, lying in...","{'next': [['329', 'Level 35']], 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Aren...",{'hires': 'https://raw.githubusercontent.com/P...,327,"[0.00905408151447773, -0.04088602587580681, -0...",0.487235
1,127,"{'chinese': '凯罗斯', 'english': 'Pinsir', 'frenc...",[Bug],"{'HP': 65, 'Attack': 125, 'Defense': 100, 'Sp....",Stag Beetle Pokémon,"Although it’s tough, it can’t handle cold well...","{'next': None, 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Mold...",{'hires': 'https://raw.githubusercontent.com/P...,126,"[0.01692618988454342, -0.03155365213751793, -0...",0.442589
2,554,"{'chinese': '火红不倒翁', 'english': 'Darumaka', 'f...",[Fire],"{'HP': 70, 'Attack': 90, 'Defense': 45, 'Sp. A...",Zen Charm Pokémon,"When it sleeps, it pulls its limbs into its bo...","{'next': [['555', 'use Ice Stone']], 'prev': N...","{'ability': [['Hustle', 'false'], ['Inner Focu...",{'hires': 'https://raw.githubusercontent.com/P...,553,"[-0.006264317315071821, -0.06521143019199371, ...",0.406497
3,722,"{'chinese': '木木枭', 'english': 'Rowlet', 'frenc...","[Grass, Flying]","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. A...",Grass Quill Pokémon,"It feels relaxed in tight, dark places and has...","{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Rea...",{'hires': 'https://raw.githubusercontent.com/P...,721,"[0.003820237936452031, -0.06739205121994019, -...",0.401396
4,775,"{'chinese': '树枕尾熊', 'english': 'Komala', 'fren...",[Normal],"{'HP': 65, 'Attack': 115, 'Defense': 65, 'Sp. ...",Drowsing Pokémon,It remains asleep from birth to death as a res...,"{'next': None, 'prev': None}","{'ability': [['Comatose', 'false']], 'egg': ['...",{'hires': 'https://raw.githubusercontent.com/P...,774,"[-0.01760929264128208, -0.03543251007795334, -...",0.371769


## Querying the collection's embeddings, eager execution
### This is useful if you don't need to chain calls together and just want to get the results back immediately

In [14]:
sleepy_pokemon = collection.query("embeddings_description", query=["Loves to sleep"], k=10, lazy=False)
sleepy_pokemon.show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,descriptionUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,embeddings_descriptionPython,embeddings_description_scoreFloat64
143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'french': 'Ronfl...",['Normal'],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp. Attack': 6...",Sleeping Pokémon,It has no interest in anything other than eating. Even if...,"{'next': None, 'prev': ['446', 'high Friendship']}","{'ability': [['Immunity', 'false'], ['Thick Fat', 'false'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,142,"<np.ndarrayshape=(768,)dtype=float64>",0.561686
25,"{'chinese': '皮卡丘', 'english': 'Pikachu', 'french': 'Pikac...",['Electric'],"{'HP': 35, 'Attack': 55, 'Defense': 40, 'Sp. Attack': 50,...",Mouse Pokémon,"While sleeping, it generates electricity in the sacs in i...","{'next': [['26', 'use Thunder Stone']], 'prev': ['172', '...","{'ability': [['Static', 'false'], ['Lightning Rod', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,24,"<np.ndarrayshape=(768,)dtype=float64>",0.474437
328,"{'chinese': '大颚蚁', 'english': 'Trapinch', 'french': 'Krak...",['Ground'],"{'HP': 45, 'Attack': 100, 'Defense': 45, 'Sp. Attack': 45...",Ant Pit Pokémon,"It builds nests that double as traps, lying in wait for i...","{'next': [['329', 'Level 35']], 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Arena Trap', 'f...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,327,"<np.ndarrayshape=(768,)dtype=float64>",0.450725
722,"{'chinese': '木木枭', 'english': 'Rowlet', 'french': 'Brindi...","['Grass', 'Flying']","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. Attack': 50,...",Grass Quill Pokémon,"It feels relaxed in tight, dark places and has been known...","{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Reach', 'true'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,721,"<np.ndarrayshape=(768,)dtype=float64>",0.445208
554,"{'chinese': '火红不倒翁', 'english': 'Darumaka', 'french': 'Da...",['Fire'],"{'HP': 70, 'Attack': 90, 'Defense': 45, 'Sp. Attack': 15,...",Zen Charm Pokémon,"When it sleeps, it pulls its limbs into its body and its ...","{'next': [['555', 'use Ice Stone']], 'prev': None}","{'ability': [['Hustle', 'false'], ['Inner Focus', 'true']...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,553,"<np.ndarrayshape=(768,)dtype=float64>",0.430829


## Performing filters

### With vexpresso we can leverage duckdb for powerful queries

#### In order to filter on metadata, you must supply a filter dictionary to the filter method. The dictionary must have the following structure:

```python
{
    <field>: {
        <filter_method>: <value>
    },
    <field>: {
        <filter_method>: <value>
    },
}

```

#### Query to only get Pokemon with the name "Snorlax"

In [15]:
filter_condition = {
    "name.english": {
        "eq":"Snorlax"
    },
}

sleepy_pokemon.filter(filter_condition).to_pandas()

,id,name,type,base,species,description,evolution,profile,image,vexpresso_index,embeddings_description,embeddings_description_score
0,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'fren...",[Normal],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp....",Sleeping Pokémon,It has no interest in anything other than eati...,"{'next': None, 'prev': ['446', 'high Friendshi...","{'ability': [['Immunity', 'false'], ['Thick Fa...",{'hires': 'https://raw.githubusercontent.com/P...,142,"[0.003054492874071002, -0.01002109982073307, -...",0.561686


#### Query to only get Pokemon with the name "Snorlax" or "Pikachu"

In [16]:
filter_condition = {
    "name.english": [
        {
            "eq":"Snorlax"
        },
        {
            "eq":"Pikachu"
        },
    ],
}
sleepy_pokemon.filter(filter_condition).to_pandas()

,id,name,type,base,species,description,evolution,profile,image,vexpresso_index,embeddings_description,embeddings_description_score
0,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'fren...",[Normal],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp....",Sleeping Pokémon,It has no interest in anything other than eati...,"{'next': None, 'prev': ['446', 'high Friendshi...","{'ability': [['Immunity', 'false'], ['Thick Fa...",{'hires': 'https://raw.githubusercontent.com/P...,142,"[0.003054492874071002, -0.01002109982073307, -...",0.561686
1,25,"{'chinese': '皮卡丘', 'english': 'Pikachu', 'fren...",[Electric],"{'HP': 35, 'Attack': 55, 'Defense': 40, 'Sp. A...",Mouse Pokémon,"While sleeping, it generates electricity in th...","{'next': [['26', 'use Thunder Stone']], 'prev'...","{'ability': [['Static', 'false'], ['Lightning ...",{'hires': 'https://raw.githubusercontent.com/P...,24,"[0.03698570281267166, -0.06595141440629959, -0...",0.474437


#### Query to only get Pokemon with the name "Snorlax" or "Pikachu" using a list

In [17]:
filter_condition = {
    "name.english": {
        "isin":["Snorlax", "Pikachu"]
    }
}
sleepy_pokemon.filter(filter_condition).to_pandas()

,id,name,type,base,species,description,evolution,profile,image,vexpresso_index,embeddings_description,embeddings_description_score
0,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'fren...",[Normal],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp....",Sleeping Pokémon,It has no interest in anything other than eati...,"{'next': None, 'prev': ['446', 'high Friendshi...","{'ability': [['Immunity', 'false'], ['Thick Fa...",{'hires': 'https://raw.githubusercontent.com/P...,142,"[0.003054492874071002, -0.01002109982073307, -...",0.561686
1,25,"{'chinese': '皮卡丘', 'english': 'Pikachu', 'fren...",[Electric],"{'HP': 35, 'Attack': 55, 'Defense': 40, 'Sp. A...",Mouse Pokémon,"While sleeping, it generates electricity in th...","{'next': [['26', 'use Thunder Stone']], 'prev'...","{'ability': [['Static', 'false'], ['Lightning ...",{'hires': 'https://raw.githubusercontent.com/P...,24,"[0.03698570281267166, -0.06595141440629959, -0...",0.474437


#### Query to only get Pokemon with type Normal and attack > 100

In [18]:
filter_condition = {
    "type": {
        "contains":"Normal"
    },
    "base.Attack": {
        "gt":100
    }
}
sleepy_pokemon.filter(filter_condition).to_pandas()

,id,name,type,base,species,description,evolution,profile,image,vexpresso_index,embeddings_description,embeddings_description_score
0,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'fren...",[Normal],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp....",Sleeping Pokémon,It has no interest in anything other than eati...,"{'next': None, 'prev': ['446', 'high Friendshi...","{'ability': [['Immunity', 'false'], ['Thick Fa...",{'hires': 'https://raw.githubusercontent.com/P...,142,"[0.003054492874071002, -0.01002109982073307, -...",0.561686
1,775,"{'chinese': '树枕尾熊', 'english': 'Komala', 'fren...",[Normal],"{'HP': 65, 'Attack': 115, 'Defense': 65, 'Sp. ...",Drowsing Pokémon,It remains asleep from birth to death as a res...,"{'next': None, 'prev': None}","{'ability': [['Comatose', 'false']], 'egg': ['...",{'hires': 'https://raw.githubusercontent.com/P...,774,"[-0.01760929264128208, -0.03543251007795334, -...",0.421351


## Query + Filter chaining

#### For example, let's filter the above collection for Pokemon with HP >= 35 and Defense > 45

In [19]:
filter_conditions = {
    "base.HP": {"gte":35},
    "base.Defense": {"gt":45}
}
sleepy_pokemon = collection.query("embeddings_description", query=["Loves to sleep"], k=10).filter(filter_conditions=filter_conditions).execute()
sleepy_pokemon.show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,descriptionUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,embeddings_descriptionPython,embeddings_description_scoreFloat64
143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'french': 'Ronfl...",['Normal'],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp. Attack': 6...",Sleeping Pokémon,It has no interest in anything other than eating. Even if...,"{'next': None, 'prev': ['446', 'high Friendship']}","{'ability': [['Immunity', 'false'], ['Thick Fat', 'false'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,142,"<np.ndarrayshape=(768,)dtype=float64>",0.561686
722,"{'chinese': '木木枭', 'english': 'Rowlet', 'french': 'Brindi...","['Grass', 'Flying']","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. Attack': 50,...",Grass Quill Pokémon,"It feels relaxed in tight, dark places and has been known...","{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Reach', 'true'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,721,"<np.ndarrayshape=(768,)dtype=float64>",0.445208
127,"{'chinese': '凯罗斯', 'english': 'Pinsir', 'french': 'Scarab...",['Bug'],"{'HP': 65, 'Attack': 125, 'Defense': 100, 'Sp. Attack': 5...",Stag Beetle Pokémon,"Although it’s tough, it can’t handle cold well. When nigh...","{'next': None, 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Mold Breaker', ...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,126,"<np.ndarrayshape=(768,)dtype=float64>",0.427454
775,"{'chinese': '树枕尾熊', 'english': 'Komala', 'french': 'Bouma...",['Normal'],"{'HP': 65, 'Attack': 115, 'Defense': 65, 'Sp. Attack': 75...",Drowsing Pokémon,It remains asleep from birth to death as a result of the ...,"{'next': None, 'prev': None}","{'ability': [['Comatose', 'false']], 'egg': ['Field'], 'g...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,774,"<np.ndarrayshape=(768,)dtype=float64>",0.421351
301,"{'chinese': '优雅猫', 'english': 'Delcatty', 'french': 'Delc...",['Normal'],"{'HP': 70, 'Attack': 65, 'Defense': 65, 'Sp. Attack': 55,...",Prim Pokémon,Delcatty sleeps anywhere it wants without keeping a perma...,"{'next': None, 'prev': ['300', 'use Moon Stone']}","{'ability': [['Cute Charm', 'false'], ['Normalize', 'fals...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,300,"<np.ndarrayshape=(768,)dtype=float64>",0.394259


## Grab fields from metadata

In [20]:
names, types, descriptions = sleepy_pokemon.select("name.english", "type", "description").to_list()
for n, t, d in zip(names, types, descriptions):
    print(f"{n}: {t} -- {d}")

Snorlax: ['Normal'] -- It has no interest in anything other than eating. Even if you climb up on its stomach while it’s napping, it doesn’t seem to mind at all!
Rowlet: ['Grass', 'Flying'] -- It feels relaxed in tight, dark places and has been known to use its Trainer’s pocket or bag as a nest.
Pinsir: ['Bug'] -- Although it’s tough, it can’t handle cold well. When night falls, it buries itself in leafage and sleeps.
Komala: ['Normal'] -- It remains asleep from birth to death as a result of the sedative properties of the leaves that form its diet.
Delcatty: ['Normal'] -- Delcatty sleeps anywhere it wants without keeping a permanent nest. If other Pokémon approach it as it sleeps, this Pokémon will never fight—it will just move away somewhere else.
Sandslash: ['Ground'] -- Thanks to its thick claws, it’s good at climbing trees. There are plenty of Sandslash that park themselves in trees and go right to sleep.
Mesprit: ['Psychic'] -- It sleeps at the bottom of a lake. Its spirit is said 

## Save collection

In [21]:
sleepy_pokemon.save("./saved_filtered_pokemon")

saving to ./saved_filtered_pokemon


## Load collection

In [22]:
loaded_collection = vexpresso.create(
    directory_or_repo_id = "saved_filtered_pokemon",
    embedding_functions = {"embeddings_description":hf_embedding_function}
)

In [23]:
loaded_collection.show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],"baseStruct[Attack: Int64, Defense: Int64, HP: Int64, Sp. Attack: Int64, Sp. Defense: Int64, Speed: Int64]",speciesUtf8,descriptionUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,embeddings_descriptionPython,embeddings_description_scoreFloat64
143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'french': 'Ronfl...",['Normal'],"{'Attack': 110, 'Defense': 65, 'HP': 160, 'Sp. Attack': 6...",Sleeping Pokémon,It has no interest in anything other than eating. Even if...,"{'next': None, 'prev': ['446', 'high Friendship']}","{'ability': [['Immunity', 'false'], ['Thick Fat', 'false'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,142,"<np.ndarrayshape=(768,)dtype=float64>",0.561686
722,"{'chinese': '木木枭', 'english': 'Rowlet', 'french': 'Brindi...","['Grass', 'Flying']","{'Attack': 55, 'Defense': 55, 'HP': 68, 'Sp. Attack': 50,...",Grass Quill Pokémon,"It feels relaxed in tight, dark places and has been known...","{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Reach', 'true'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,721,"<np.ndarrayshape=(768,)dtype=float64>",0.445208
127,"{'chinese': '凯罗斯', 'english': 'Pinsir', 'french': 'Scarab...",['Bug'],"{'Attack': 125, 'Defense': 100, 'HP': 65, 'Sp. Attack': 5...",Stag Beetle Pokémon,"Although it’s tough, it can’t handle cold well. When nigh...","{'next': None, 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Mold Breaker', ...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,126,"<np.ndarrayshape=(768,)dtype=float64>",0.427454
775,"{'chinese': '树枕尾熊', 'english': 'Komala', 'french': 'Bouma...",['Normal'],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. Attack': 75...",Drowsing Pokémon,It remains asleep from birth to death as a result of the ...,"{'next': None, 'prev': None}","{'ability': [['Comatose', 'false']], 'egg': ['Field'], 'g...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,774,"<np.ndarrayshape=(768,)dtype=float64>",0.421351
301,"{'chinese': '优雅猫', 'english': 'Delcatty', 'french': 'Delc...",['Normal'],"{'Attack': 65, 'Defense': 65, 'HP': 70, 'Sp. Attack': 55,...",Prim Pokémon,Delcatty sleeps anywhere it wants without keeping a perma...,"{'next': None, 'prev': ['300', 'use Moon Stone']}","{'ability': [['Cute Charm', 'false'], ['Normalize', 'fals...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,300,"<np.ndarrayshape=(768,)dtype=float64>",0.394259


## Uploading to HFhub

In [24]:
# Automatically gets token from env variable
# HUGGINGFACEHUB_API_TOKEN = ...
# or you can pass in token directly via collection.save(token=...)
# repo_id = {HF_USERNAME}/{REPO_NAME}
repo_id = "shyamsn97/pokemon_vexpresso_test"
loaded_collection.save(repo_id, to_hub=True, private=True)

Uploading collection to shyamsn97/pokemon_vexpresso_test


Upload 1 LFS files:   0%|                                                         | 0/1 [00:00<?, ?it/s]
content.parquet:   0%|                                                      | 0.00/77.1k [00:00<?, ?B/s]
content.parquet: 100%|██████████████████████████████████████████████| 77.1k/77.1k [00:00<00:00, 106kB/s]
Upload 1 LFS files: 100%|█████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.00it/s]


Upload to shyamsn97/pokemon_vexpresso_test complete!


'shyamsn97/pokemon_vexpresso_test'

In [25]:
# or upload with username + repo name
loaded_collection.save(hf_username = "shyamsn97", repo_name = "pokemon_vexpresso_test", to_hub=True)

Uploading collection to None
Upload to shyamsn97/pokemon_vexpresso_test complete!


'shyamsn97/pokemon_vexpresso_test'

## Load from HfHub

In [26]:
# Automatically gets token from env variable
# HUGGINGFACEHUB_API_TOKEN = ...
# or you can pass in token directly via Collection.load(token=...)
# repo_id = {HF_USERNAME}/{REPO_NAME}
repo_id = "shyamsn97/pokemon_vexpresso_test"

collection = vexpresso.create(
    directory_or_repo_id = repo_id,
    embedding_functions = {"embeddings_description":hf_embedding_function}
)
collection.to_pandas()

Retrieving from hf repo: shyamsn97/pokemon_vexpresso_test


Fetching 2 files: 100%|███████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.57it/s]


,id,name,type,base,species,description,evolution,profile,image,vexpresso_index,embeddings_description,embeddings_description_score
0,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'fren...",[Normal],"{'Attack': 110, 'Defense': 65, 'HP': 160, 'Sp....",Sleeping Pokémon,It has no interest in anything other than eati...,"{'next': None, 'prev': ['446', 'high Friendshi...","{'ability': [['Immunity', 'false'], ['Thick Fa...",{'hires': 'https://raw.githubusercontent.com/P...,142,"[0.003054492874071002, -0.01002109982073307, -...",0.561686
1,722,"{'chinese': '木木枭', 'english': 'Rowlet', 'frenc...","[Grass, Flying]","{'Attack': 55, 'Defense': 55, 'HP': 68, 'Sp. A...",Grass Quill Pokémon,"It feels relaxed in tight, dark places and has...","{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Rea...",{'hires': 'https://raw.githubusercontent.com/P...,721,"[0.003820237936452031, -0.06739205121994019, -...",0.445208
2,127,"{'chinese': '凯罗斯', 'english': 'Pinsir', 'frenc...",[Bug],"{'Attack': 125, 'Defense': 100, 'HP': 65, 'Sp....",Stag Beetle Pokémon,"Although it’s tough, it can’t handle cold well...","{'next': None, 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Mold...",{'hires': 'https://raw.githubusercontent.com/P...,126,"[0.01692618988454342, -0.03155365213751793, -0...",0.427454
3,775,"{'chinese': '树枕尾熊', 'english': 'Komala', 'fren...",[Normal],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. ...",Drowsing Pokémon,It remains asleep from birth to death as a res...,"{'next': None, 'prev': None}","{'ability': [['Comatose', 'false']], 'egg': ['...",{'hires': 'https://raw.githubusercontent.com/P...,774,"[-0.01760929264128208, -0.03543251007795334, -...",0.421351
4,301,"{'chinese': '优雅猫', 'english': 'Delcatty', 'fre...",[Normal],"{'Attack': 65, 'Defense': 65, 'HP': 70, 'Sp. A...",Prim Pokémon,Delcatty sleeps anywhere it wants without keep...,"{'next': None, 'prev': ['300', 'use Moon Stone']}","{'ability': [['Cute Charm', 'false'], ['Normal...",{'hires': 'https://raw.githubusercontent.com/P...,300,"[0.01472731214016676, 0.012978807091712952, -0...",0.394259
5,28,"{'chinese': '穿山王', 'english': 'Sandslash', 'fr...",[Ground],"{'Attack': 100, 'Defense': 110, 'HP': 75, 'Sp....",Mouse Pokémon,"Thanks to its thick claws, it’s good at climbi...","{'next': None, 'prev': ['27', 'Level 22']}","{'ability': [['Sand Veil', 'false'], ['Sand Ru...",{'hires': 'https://raw.githubusercontent.com/P...,27,"[-0.0045143356546759605, -0.049847312271595, 0...",0.390875
6,481,"{'chinese': '艾姆利多', 'english': 'Mesprit', 'fre...",[Psychic],"{'Attack': 105, 'Defense': 105, 'HP': 80, 'Sp....",Emotion Pokémon,It sleeps at the bottom of a lake. Its spirit ...,"{'next': None, 'prev': None}","{'ability': [['Levitate', 'false']], 'egg': ['...",{'hires': 'https://raw.githubusercontent.com/P...,480,"[0.04477566480636597, -0.02245207130908966, -0...",0.387243
7,224,"{'chinese': '章鱼桶', 'english': 'Octillery', 'fr...",[Water],"{'Attack': 105, 'Defense': 75, 'HP': 75, 'Sp. ...",Jet Pokémon,It usually sleeps in caves. The pitch-black in...,"{'next': None, 'prev': ['223', 'Level 25']}","{'ability': [['Suction Cups', 'false'], ['Snip...",{'hires': 'https://raw.githubusercontent.com/P...,223,"[0.07503223419189453, -0.032861337065696716, -...",0.384575
8,575,"{'chinese': '哥德小童', 'english': 'Gothorita', 'f...",[Psychic],"{'Attack': 45, 'Defense': 70, 'HP': 60, 'Sp. A...",Manipulate Pokémon,"According to many old tales, it creates friend...","{'next': [['576', 'Level 41']], 'prev': ['574'...","{'ability': [['Frisk', 'false'], ['Competitive...",{'hires': 'https://raw.githubusercontent.com/P...,574,"[0.06230613589286804, -0.01704530604183674, -0...",0.370455
9,97,"{'chinese': '引梦貘人', 'english': 'Hypno', 'frenc...",[Psychic],"{'Attack': 73, 'Defense': 70, 'HP': 85, 'Sp. A...",Hypnosis Pokémon,There are some Hypno that assist doctors with ...,"{'next': None, 'prev': ['96', 'Level 26']}","{'ability': [['Insomnia', 'false'], ['Forewarn...",{'hires': 'ht

In [28]:
# or load with username + repo name
collection = vexpresso.create(
    hf_username = "shyamsn97",
    repo_name = "pokemon_vexpresso_test",
    embedding_functions = {"embeddings_description":hf_embedding_function}
)
collection.to_pandas()

Retrieving from hf repo: shyamsn97/pokemon_vexpresso_test


Fetching 2 files: 100%|█████████████████████████████████████████████████| 2/2 [00:00<00:00, 4556.55it/s]


,id,name,type,base,species,description,evolution,profile,image,vexpresso_index,embeddings_description,embeddings_description_score
0,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'fren...",[Normal],"{'Attack': 110, 'Defense': 65, 'HP': 160, 'Sp....",Sleeping Pokémon,It has no interest in anything other than eati...,"{'next': None, 'prev': ['446', 'high Friendshi...","{'ability': [['Immunity', 'false'], ['Thick Fa...",{'hires': 'https://raw.githubusercontent.com/P...,142,"[0.003054492874071002, -0.01002109982073307, -...",0.561686
1,722,"{'chinese': '木木枭', 'english': 'Rowlet', 'frenc...","[Grass, Flying]","{'Attack': 55, 'Defense': 55, 'HP': 68, 'Sp. A...",Grass Quill Pokémon,"It feels relaxed in tight, dark places and has...","{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Rea...",{'hires': 'https://raw.githubusercontent.com/P...,721,"[0.003820237936452031, -0.06739205121994019, -...",0.445208
2,127,"{'chinese': '凯罗斯', 'english': 'Pinsir', 'frenc...",[Bug],"{'Attack': 125, 'Defense': 100, 'HP': 65, 'Sp....",Stag Beetle Pokémon,"Although it’s tough, it can’t handle cold well...","{'next': None, 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Mold...",{'hires': 'https://raw.githubusercontent.com/P...,126,"[0.01692618988454342, -0.03155365213751793, -0...",0.427454
3,775,"{'chinese': '树枕尾熊', 'english': 'Komala', 'fren...",[Normal],"{'Attack': 115, 'Defense': 65, 'HP': 65, 'Sp. ...",Drowsing Pokémon,It remains asleep from birth to death as a res...,"{'next': None, 'prev': None}","{'ability': [['Comatose', 'false']], 'egg': ['...",{'hires': 'https://raw.githubusercontent.com/P...,774,"[-0.01760929264128208, -0.03543251007795334, -...",0.421351
4,301,"{'chinese': '优雅猫', 'english': 'Delcatty', 'fre...",[Normal],"{'Attack': 65, 'Defense': 65, 'HP': 70, 'Sp. A...",Prim Pokémon,Delcatty sleeps anywhere it wants without keep...,"{'next': None, 'prev': ['300', 'use Moon Stone']}","{'ability': [['Cute Charm', 'false'], ['Normal...",{'hires': 'https://raw.githubusercontent.com/P...,300,"[0.01472731214016676, 0.012978807091712952, -0...",0.394259
5,28,"{'chinese': '穿山王', 'english': 'Sandslash', 'fr...",[Ground],"{'Attack': 100, 'Defense': 110, 'HP': 75, 'Sp....",Mouse Pokémon,"Thanks to its thick claws, it’s good at climbi...","{'next': None, 'prev': ['27', 'Level 22']}","{'ability': [['Sand Veil', 'false'], ['Sand Ru...",{'hires': 'https://raw.githubusercontent.com/P...,27,"[-0.0045143356546759605, -0.049847312271595, 0...",0.390875
6,481,"{'chinese': '艾姆利多', 'english': 'Mesprit', 'fre...",[Psychic],"{'Attack': 105, 'Defense': 105, 'HP': 80, 'Sp....",Emotion Pokémon,It sleeps at the bottom of a lake. Its spirit ...,"{'next': None, 'prev': None}","{'ability': [['Levitate', 'false']], 'egg': ['...",{'hires': 'https://raw.githubusercontent.com/P...,480,"[0.04477566480636597, -0.02245207130908966, -0...",0.387243
7,224,"{'chinese': '章鱼桶', 'english': 'Octillery', 'fr...",[Water],"{'Attack': 105, 'Defense': 75, 'HP': 75, 'Sp. ...",Jet Pokémon,It usually sleeps in caves. The pitch-black in...,"{'next': None, 'prev': ['223', 'Level 25']}","{'ability': [['Suction Cups', 'false'], ['Snip...",{'hires': 'https://raw.githubusercontent.com/P...,223,"[0.07503223419189453, -0.032861337065696716, -...",0.384575
8,575,"{'chinese': '哥德小童', 'english': 'Gothorita', 'f...",[Psychic],"{'Attack': 45, 'Defense': 70, 'HP': 60, 'Sp. A...",Manipulate Pokémon,"According to many old tales, it creates friend...","{'next': [['576', 'Level 41']], 'prev': ['574'...","{'ability': [['Frisk', 'false'], ['Competitive...",{'hires': 'https://raw.githubusercontent.com/P...,574,"[0.06230613589286804, -0.01704530604183674, -0...",0.370455
9,97,"{'chinese': '引梦貘人', 'english': 'Hypno', 'frenc...",[Psychic],"{'Attack': 73, 'Defense': 70, 'HP': 85, 'Sp. A...",Hypnosis Pokémon,There are some Hypno that assist doctors with ...,"{'next': None, 'prev': ['96', 'Level 26']}","{'ability': [['Insomnia', 'false'], ['Forewarn...",{'hires': 'ht